In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from mne.filter import filter_data
from scipy.signal import find_peaks_cwt
from entropy import shannon_entropy, sample_entropy
import time

In [4]:
trialData = np.load('C:/data/processed/ESM_pilot/trials.npy')
esm = pd.read_csv('C:\data\processed\ESM_pilot\esm.csv')

In [150]:
alignedFeatures = extractFeatures(trialData,esm,100,windowLength=60)
alignedFeatures.to_csv('C:/data/processed/ESM_pilot/features.csv',index=False)

C:\ProgramData\Miniconda3\envs\p3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


1.0391898155212402
1.057173728942871
1.05118989944458
1.0212700366973877
1.006310224533081
1.0003249645233154
1.0043151378631592
1.04620361328125
1.0063090324401855
1.0352327823638916
1.0103001594543457
1.0781188011169434
1.05617356300354
1.053166151046753
1.0202739238739014
1.0551791191101074
1.0053107738494873
1.0093042850494385
0.9943413734436035
1.0471982955932617
1.0192747116088867
1.0372273921966553
1.0073070526123047
1.0442357063293457
1.0441803932189941
1.051189661026001
1.1160168647766113
1.2606287002563477
1.1319761276245117
1.0083038806915283
1.1768524646759033
0.9913511276245117
1.0023186206817627
0.9953243732452393
0.9763913154602051
1.095097541809082
1.0731046199798584
1.0023205280303955
0.9674139022827148
1.0242629051208496
0.9903514385223389
0.9704077243804932
0.9973340034484863
1.0601630210876465
0.9813790321350098
0.9993257522583008
0.9763898849487305
1.0262563228607178
1.0003254413604736
0.99334716796875
1.0122921466827393
0.9983041286468506
1.013291597366333
0.97240

In [149]:
def extractFeatures(data, esm,sr, windowLength=60):

    numSamples=data.shape[0]
    # Getting number and names of features
    tremorNames, _ = tremorFeatures(data[0,:windowLength*sr,:], sr,windowLength=windowLength)
    bradyNames, _ = bradykinesiaFeatures(data[0,:windowLength*sr,:], sr,windowLength=windowLength)
    cols=[]
    for s in ['L','R','C']:
        cols.extend([c + s for c in tremorNames])
        cols.extend([c + s for c in bradyNames])
    cols.extend(esm.keys())
    aligned = pd.DataFrame(columns=cols)
    accelerometerChannel = [a + b for a in  [0,6,12] for b in range(3)]
    for beep in range(data.shape[0]):
        t=time.time()
        allFeat = []
        numWindows = int(data.shape[1]/sr/windowLength)
        buff = data[beep,:,:]
        buff[:,accelerometerChannel] = (buff[:,accelerometerChannel].T - np.mean(buff[:,accelerometerChannel].T,axis=0)).T
        buff[:,accelerometerChannel] = filter_data(buff[:,accelerometerChannel].T,sr,0,3,method='iir',verbose='WARNING').T
        for s,sID in enumerate([range(6),range(6,12),range(12,18)]):
            
            features=np.zeros((numWindows,len(tremorNames)+len(bradyNames)))
            for i in range(0,numWindows):
                win = i * windowLength * sr
                _, features[i,:len(tremorNames)] = tremorFeatures(buff[win:win+windowLength*sr,sID],sr,windowLength=windowLength)
                _, features[i,len(tremorNames):] = bradykinesiaFeatures(buff[win:win+windowLength*sr,sID],sr,windowLength=windowLength)
            allFeat.append(features)
        allFeat = np.concatenate(allFeat,axis=1)
        allFeat = np.concatenate((allFeat, np.matlib.repmat(esm.iloc[beep,:],numWindows,1)),axis=1)
        aligned = aligned.append(pd.DataFrame(allFeat,columns = cols),ignore_index=True)
        print(time.time()-t)
        # Add the power between 3.6 and 9.4
        #Timestamp at beginning of each window
        #alignedTimes.append(startTime + pd.Timedelta('%d s ' % (i * windowLength)))
    return aligned

In [146]:
def tremorFeatures(windowData,sr,windowLength=60):
    gyroChannel={'X':3,'Y':4,'Z':5} # gyro is xyz 3-4-5
    if windowData.shape[0]!=sr*windowLength:
        print(windowData.shape,sr*windowLength)
    freq = np.fft.rfftfreq(windowLength*sr, d=1./sr)
    selected=np.logical_and(freq>3.5,freq<7.5)
    features=[]
    featureNames=[]
    for ch in gyroChannel.keys():
        spec = np.log(np.sum(np.abs(np.fft.rfft(windowData[:,gyroChannel[ch]]))[selected]))
        features.append(spec)
        featureNames.append('BandPower' + ch)
    return featureNames, features

In [145]:
def bradykinesiaFeatures(windowData,sr,windowLength=60):
    features=[]
    featureNames=[]
    accelerometerChannel={'X':0,'Y':1,'Z':2} # assuming acc xyz 0-1-2 is
    
    

    # lowpass filter signal <3 Hz
    
    # Features: (Patel et al IEEE 2009)
    # rms
    # range
    # entropy
    # normalized cross-correlation value and time point
    # dominat frequency and ratio between dominant and rest
    freq = np.fft.rfftfreq(windowLength*sr, d=1./sr)
    
    for ch in accelerometerChannel.keys():
        #ent = shannon_entropy(windowData[:,accelerometerChannel[ch]])
        #features.append(ent)
        #featureNames.append('Entropy' + ch)
        
        spec = np.abs(np.fft.rfft(windowData[:,accelerometerChannel[ch]]))
        domFreq = freq[np.argmax(spec)]
        features.append(domFreq)
        featureNames.append('DomFreq' + ch)
        
        domEnergyRatio = np.max(spec) / np.sum(spec)
        features.append(domEnergyRatio)
        featureNames.append('DomEnergyRatio' + ch)
        
        rms = np.sqrt(np.mean(windowData[:,accelerometerChannel[ch]]**2))
        features.append(rms)
        featureNames.append('RMS' + ch)
        
        ampRange = np.max(windowData[:,accelerometerChannel[ch]]) - np.min(windowData[:,accelerometerChannel[ch]])
        features.append(ampRange)
        featureNames.append('AmpRange' + ch)
    
    cCMax=[]
    cCLocs=[]
    for i, ch1 in enumerate(accelerometerChannel.keys()):
        for j,ch2 in enumerate(list(accelerometerChannel.keys())[i+1:]):
            crossCorr = np.correlate(windowData[:,accelerometerChannel[ch1]],windowData[:,accelerometerChannel[ch2]],'same')
            crossCorr = crossCorr/(np.std(windowData[:,accelerometerChannel[ch1]]) * np.std(windowData[:,accelerometerChannel[ch2]]))
            
            cCMax.append(np.max(crossCorr))
            cCLocs.append(np.argmax(crossCorr))
    features.append(np.max(cCMax))
    featureNames.append('MaxCC')
    features.append(cCLocs[np.argmax(cCMax)])
    featureNames.append('MaxCCLoc')
        
        #peaks=find_peaks_cwt(windowData[accelerometerChannel[ch],:],np.arange(1,10))
        #peaks=[1,2,3]
        #features.append(len(peaks))
        #featureNames.append('#Movements' + ch)
        
    #features.append(np.max(windowData[0:3,:]))
    #featureNames.append('MaxMovement')
    return featureNames, features